In [20]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from PIL import Image
from pathlib import Path
import pandas as pd
import os
from padfunc import CustomResizeAndReflectPad
import json

In [21]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [22]:
with open("C:/Data/Python/IFCBclassify_MN/data/models/model_20240624_154622/class_mapping.json", 'r') as f:    
    class_mapping = json.load(f)
class_names = list(class_mapping.keys())

In [23]:
# Transformation to be applied to the images
simple_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    CustomResizeAndReflectPad(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [24]:
# Load the model
def load_model(MODEL_PATH, class_names):
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1).to(device)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(num_ftrs, len(class_names))
    ).to(device)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model.eval()
    return model

# Predict function
def predict_image(image_path, model, transform, class_names):
    image = Image.open(image_path)
    transformed_image = transform(image).unsqueeze(0).to(device)
    
    model.eval()
    with torch.inference_mode():
        pred_logit = model(transformed_image)
        predicted_prob = torch.softmax(pred_logit, dim=1)
        pred_label = torch.argmax(predicted_prob, dim=1).item()
        predicted_class = class_names[pred_label]
        predicted_prob = predicted_prob[0, pred_label].item()
        return predicted_class, predicted_prob


# Predict on all images in a folder
def predict_folder(folder_path, model, transform, class_names, output_file):
    results = []
    image_paths = list(Path(folder_path).glob('*.png'))

    for image_path in image_paths:
        predicted_class, predicted_prob = predict_image(image_path, model, transform, class_names)
        results.append([image_path.name, predicted_class, predicted_prob])

    # Save results to a CSV file
    df = pd.DataFrame(results, columns=['image_name', 'predicted_class', 'probability'])
    df.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")


In [25]:
#    with torch.no_grad():
        # output = model(image)
        # probabilities = torch.nn.functional.softmax(output[0], dim=0)
        # _, predicted = torch.max(output, 1)
        # predicted_class = class_names[predicted]
        # predicted_prob = probabilities[predicted].item()
        # return predicted_class, predicted_prob

In [26]:
# Set paths
MODEL_PATH = "C:/Data/Python/IFCBclassify_MN/data/models/model_20240624_154622/model.pth"
IMAGE_FOLDER = "C:/Data/Python/IFCB/data/Diverse/"  # Ensure this is the directory containing images
OUTPUT_FILE = "C:/Data/Python/IFCBclassify_MN/data/models/model_20240624_154622/predictions.csv"

# Load the model
model = load_model(MODEL_PATH, class_names)

# Predict on all images in the folder and save results
#predict_folder(IMAGE_FOLDER, model, simple_transform, CLASS_NAMES, OUTPUT_FILE)